In [ ]:
df = pd.read_csv("final.csv", sep=';', on_bad_lines='skip')
texts = df['text'].apply(preprocess)
labels = df['type'].values  # type 컬럼에서 0, 1, 2 라벨 추출

# 파라미터 설정
vocab_size = 20000  # 사전 크기 (단어 수)
max_length = 50    # 각 문장의 최대 길이
embedding_dim = 500  # 임베딩 벡터 크기

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,      # 단어 집합 크기 제한
    output_mode='int',          # 정수 인덱스로 변환
    output_sequence_length=max_length  # 패딩을 포함한 최대 길이 설정
)

class_weights = {
    0: 1.,
    1: 0.3,
    2: 10.
}

# 텍스트에 인덱스 부여
vectorize_layer.adapt(texts)
# 텍스트 데이터를 인덱스로 벡터화
texts_vectorized = vectorize_layer(texts).numpy()
# 라벨을 one-hot 인코딩으로 변환 - 0: not cyberbullying, 1: cyberbullying, 2: sexual cyberbullying
labels_categorical = to_categorical(labels, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(
    texts_vectorized, labels_categorical, test_size=0.2, random_state=42
)

# CNN 모델 설계
model = Sequential()
# 임베딩 레이어: 단어 인덱스를 임베딩 벡터로 변환
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))

# LSTM 레이어 추가
model.add(LSTM(128, return_sequences=False))  # 128개의 뉴런으로 출력
# 출력 레이어: 3개의 클래스에 대한 출력, softmax로 확률값 출력
model.add(Dense(3, activation='softmax'))
# 모델 컴파일 (손실 함수: 다중 클래스 분류를 위한 categorical_crossentropy)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 모델 요약 출력
model.summary()
# 학습
history = model.fit(X_train, y_train, epochs=20, batch_size=500, class_weight=class_weights, shuffle=True)

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")